In [1]:
from geocal import *
from ecostress_swig import *
import h5py
import os

In [2]:
orb_fname = "/data/smyth/ecostress-test-data/latest/L1A_RAW_ATT_80005_20150124T204251_0100_01.h5.expected"
orb = HdfOrbit_Eci_TimeJ2000(orb_fname, "", "Ephemeris/time_j2000",
                             "Ephemeris/eci_position",
                             "Ephemeris/eci_velocity",
                             "Attitude/time_j2000",
                             "Attitude/quaternion")
rad_fname = "/data/smyth/ecostress-test-data/latest/ECOSTRESS_L1B_RAD_80005_001_20150124T204251_0100_01.h5.expected"
f = h5py.File(rad_fname, "r")
tmlist = f["/Time/line_start_time_j2000"][:]
tt = MeasuredTimeTable([Time.time_j2000(t) for t in tmlist])
datum = os.environ["AFIDS_VDEV_DATA"] + "/EGM96_20_x100.HLF"
srtm_dir = os.environ["ELEV_ROOT"]
dem = SrtmDem(srtm_dir,False, DatumGeoid96(datum))

In [3]:
# Initial camera, we will change this
cam_fname = "/data/smyth/ecostress-test-data/latest/l1_osp_dir/camera.xml"
cam_old = read_shelve(cam_fname)

In [4]:
def pixel_size(cam, ic):
    t, fc = tt.time(ic)
    od = orb.orbit_data(t)
    gp1 = od.surface_intersect(cam, fc, dem)
    fc.line = fc.line + 1
    gp2 = od.surface_intersect(cam, fc, dem)
    fc.line = fc.line - 1
    fc.sample = fc.sample + 1
    gp3 = od.surface_intersect(cam, fc, dem)
    print(distance(gp1, gp2))
    print(distance(gp1, gp3))

In [5]:
pixel_size(cam_old, ImageCoordinate(0,0))

70.38688759623848
76.768843193277


In [6]:
cam = QuaternionCamera(Quaternion_double(1,0,0,0), 256, 1, 
                       40e-3,40e-3,425, FrameCoordinate(128,0.5))

In [7]:
pixel_size(cam, ImageCoordinate(0,0))

38.86684765419074
42.22235250917164


In [8]:
print(cam_old)

QuaternionCamera:
   Number line:     1
   Number sample:   5400
   Focal length:    425 mm
   Line pitch:      0.072 mm
   Sample pitch:    0.08 mm
   Principal point: Frame Coordinate: (0.5, 2700)
   Frame convention: LINE_IS_X
   Frame to spacecraft: (1,0,0,0)



In [9]:
print(cam)

QuaternionCamera:
   Number line:     256
   Number sample:   1
   Focal length:    425 mm
   Line pitch:      0.04 mm
   Sample pitch:    0.04 mm
   Principal point: Frame Coordinate: (128, 0.5)
   Frame convention: LINE_IS_X
   Frame to spacecraft: (1,0,0,0)



In [10]:
cam = EcostressCamera()

In [11]:
pixel_size(cam, ImageCoordinate(0,0))

38.86684765419074
42.22235250917164


In [12]:
print(isinstance(cam_old,QuaternionCamera))

True


In [13]:
print(isinstance(cam,QuaternionCamera))

True


In [14]:
print(isinstance(cam,Camera))

True


In [15]:
print(cam)

EcostressCamera


In [16]:
t, fc = tt.time(ImageCoordinate(0,0))
od = orb.orbit_data(t)
ras = MemoryRasterImage(cam.number_line(0), cam.number_sample(0))
igc = OrbitDataImageGroundConnection(od, cam, dem, ras)

In [17]:
print(igc.camera)
igc.camera

EcostressCamera


<ecostress_swig.ecostress_camera.EcostressCamera; proxy of <Swig Object of type 'boost::shared_ptr< Ecostress::EcostressCamera > *' at 0x7f5d02cd3300> >